# Feature engineering and model building

## feature engineering

In [14]:
def Moving_average(series,n):
    return series.rolling(n,min_periods=1).mean()

def divergence(series, window=25):
    std = series.rolling(window, min_periods=1).std()
    mean = series.rolling(window, min_periods=1).mean()
    return (series - mean) / std
    
def rsi(series, n=14):
    return (series - series.shift(1)).rolling(n).apply(lambda s:s[s>0].sum()/abs(s).sum())
    
def Daily_returns(df):
    return (df['Close'] - df['Open']) / df['Open']

def Normalized_volume(n):
    return df['Volume'] / df['Volume'].rolling(n,min_periods=1).mean()


def DMA(series, window=25):
    return series/Moving_average(series, window) - 1

In [15]:
'''The stochastic function calculates the Stochastic Oscillator, a momentum indicator used in technical analysis 
to determine the relative position of the current closing price compared to its price range over a given period.
The function also computes two smoothed versions of this indicator, 
often referred to as the Fast %D and Slow %D lines, which are used for trend analysis and trading signals.'''
def stochastic(series, k=14, n=3, m=3):
    _min = series.rolling(k).min()
    _max = series.rolling(k).max()
    _k = (series - _min)/(_max - _min)
    _d1 = _k.rolling(n).mean()
    _d2 = _d1.rolling(m).mean()
    return pd.DataFrame({
                    "%K":_k,
                    "FAST-%D":_d1,
                    "SLOW-%D":_d2,
                    },index=series.index)
def psy(series, n=14):
    return (series - series.shift(1)).rolling(n).apply(lambda s:(s>=0).mean())

def ICH(series):
    conv = series.rolling(9).apply(lambda s:(s.max()+s.min())/2)
    base = series.rolling(26).apply(lambda s:(s.max()+s.min())/2)
    pre1 = ((conv + base)/2).shift(25)
    pre2 = d.Close_adj.rolling(52).apply(lambda s:(s.max()+s.min())/2).shift(25)
    lagg = d.Close_adj.shift(25)
    return conv, base, pre1, pre2, lagg

def roc(series, window=14):
    return series/series.shift(window) - 1
    
class FeatureBase():
    def create_feature(self,d):
        assert False, "NotImplemented"
    

In [16]:
class MAFeature(FeatureBase):
    def create_feature(self, d):
        return self._create_feature(d["Close_adj"])

    def _create_feature(self, series, window1=5, window2=25):
        ma1 = Moving_average(series, window1).rename("MA1")
        ma2 = Moving_average(series, window2).rename("MA2")
        diff = ma1 - ma2
        cross = pd.Series(
                        np.where((diff>0) & (diff<0).shift().fillna(False), 1,
                            np.where((diff<0) & (diff>0).shift().fillna(False), -1, 0
                                )
                        ),
                        index = series.index, name="MA_Cross"
                )
        return pd.concat([ma1, ma2, cross], axis=1)


In [17]:
import datetime
def holiday(d):
    # Ensure 'Date' is a datetime column
    d['Date'] = pd.to_datetime(d['Date'])
    
    #  the 'weekday' column (0 = Monday, 6 = Sunday)
    d['weekday'] = d['Date'].dt.weekday
    
    # Creating the before_holiday and after_holiday columns
    return pd.DataFrame({
        "before_holiday": (d["Date"] != d["Date"].shift(-1) - datetime.timedelta(days=1)) | (d["weekday"] == 4),
        "after_holiday": (d["Date"] != d["Date"].shift(1) + datetime.timedelta(days=1)) | (d["weekday"] == 0)
    }, index=d.index)


In [18]:
def make_features(df):
    df=df[[
        "Date","SecuritiesCode","Open","Close","AdjustmentFactor",
        "Volume"
    ]].copy()
    df["weekday"] = df["Date"].dt.weekday
    holiday_info = holiday(df)
    df = df.join(holiday_info)
   
    # Volume ratio indicates unusual trading activity by comapring it to the average volume of the last 15 days
    df["Volume_ratio"] = df["Volume"]/df.groupby("SecuritiesCode")["Volume"].rolling(window=15, min_periods=1).mean().reset_index("SecuritiesCode",drop=True)

    # Adjusted closing price to account for stock splits,dividend and other corporate actions by dividing the raw price by the cumulative adjustment factor
    df["Close_adj"] = (df.groupby("SecuritiesCode")[["Close", "AdjustmentFactor"]].apply(lambda d: d["Close"] / d["AdjustmentFactor"].cumprod().shift().fillna(1))
        .reset_index(level=0, drop=True))

    df[["MA1", "MA2", "MA_Cross"]] = (df.groupby("SecuritiesCode").apply(lambda d: MAFeature()._create_feature(d["Close_adj"]), include_groups=False)
    .reset_index(level=0, drop=True))
    
    df["Diff"] = (df["Close"] - df["Open"]) / df[["Close","Open"]].mean(axis=1)
    df["Diff_MA1"] = df["Close_adj"] - df["MA1"]
    df["Diff_MA2"] = df["Close_adj"] - df["MA2"]
    for i in range(1, 3):
        df["MA_Cross_lag_{:}".format(i)] = df.groupby("SecuritiesCode")["MA_Cross"].shift(i)
        
    df["DivMA"] = df.groupby("SecuritiesCode")["Close_adj"].apply(DMA).reset_index(level=0, drop=True)
    df["Div"] = df.groupby("SecuritiesCode")["Close_adj"].apply(divergence).reset_index(level=0, drop=True)
    df["Rsi"] = df.groupby("SecuritiesCode")["Close_adj"].apply(rsi).reset_index(level=0, drop=True)
    df = df.join(df.groupby("SecuritiesCode")["Close_adj"].apply(stochastic).reset_index(level=0, drop=True))

    return df

In [19]:
df=make_features(stocks)

/tmp/ipykernel_24/3243037919.py:10: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  np.where((diff>0) & (diff<0).shift().fillna(False), 1,
/tmp/ipykernel_24/3243037919.py:11: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  np.where((diff<0) & (diff>0).shift().fillna(False), -1, 0


In [20]:
df.columns

Index(['Date', 'SecuritiesCode', 'Open', 'Close', 'AdjustmentFactor', 'Volume',
       'weekday', 'before_holiday', 'after_holiday', 'Volume_ratio',
       'Close_adj', 'MA1', 'MA2', 'MA_Cross', 'Diff', 'Diff_MA1', 'Diff_MA2',
       'MA_Cross_lag_1', 'MA_Cross_lag_2', 'DivMA', 'Div', 'Rsi', '%K',
       'FAST-%D', 'SLOW-%D'],
      dtype='object')

## Building a machine learnig model and training our model

### Prepare the Feature Set

In [51]:
df["Target"] = df.groupby("SecuritiesCode")["Close_adj"].shift(-1) / df["Close_adj"] - 1
# we want to predict next-day return based on today’s features

0         0.031639
1        -0.056027
2         0.013462
3         0.032680
4         0.032568
            ...   
261892    0.036298
261893    0.027743
261894    0.009993
261895    0.006051
261896    0.027656
Name: Target, Length: 260566, dtype: float64

In [53]:
target_col = "Target"
feature_cols = [col for col in df.columns if col != target_col and col != "Date"]


In [54]:
df['Date']

0        2021-12-06
1        2021-12-06
2        2021-12-06
3        2021-12-06
4        2021-12-06
            ...    
261892   2022-06-20
261893   2022-06-20
261894   2022-06-20
261895   2022-06-20
261896   2022-06-20
Name: Date, Length: 260566, dtype: datetime64[ns]

In [56]:
X_train = train_df[feature_cols]
y_train = train_df[target_col]

X_test = test_df[feature_cols]
y_test = test_df[target_col]

In [57]:
print(X_train.shape)
print(y_train.shape)


(155302, 24)
(155302,)


### Train a model 

In [58]:
from lightgbm import LGBMRegressor

model = LGBMRegressor(n_estimators=500, learning_rate=0.01)
model.fit(X_train, y_train)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.012681 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4360
[LightGBM] [Info] Number of data points in the train set: 155302, number of used features: 24
[LightGBM] [Info] Start training from score -0.000096


LGBMRegressor(learning_rate=0.01, n_estimators=500)

In [59]:
test_df = test_df.copy()
test_df["pred"] = model.predict(X_test)

# Rank stocks daily
test_df["rank"] = test_df.groupby("Date")["pred"].rank(ascending=False)

In [60]:
daily_scores = []

for date, group in test_df.groupby("Date"):
    top_200 = group.nsmallest(200, "rank")
    bottom_200 = group.nlargest(200, "rank")
    score = top_200["Target"].mean() - bottom_200["Target"].mean()
    daily_scores.append(score)

final_score = np.mean(daily_scores)
print("Average daily score (Top200 - Bottom200):", final_score)


Average daily score (Top200 - Bottom200): 0.0018035032621285983


###  An average daily score of 0.0018 means that, on average, Top 200 stocks outperform Bottom 200 stocks by 0.18% per day.

### Ranked stocks

In [62]:
test_df["rank"].sort_values()

196618       1.0
238427       1.0
258385       1.0
242719       1.0
256113       1.0
           ...  
234656    1989.0
159789    1989.0
214969    1990.0
182219    1990.0
183912    1991.0
Name: rank, Length: 105264, dtype: float64